# 04_ab_analysis.ipynb - A/B аналіз push-груп

## 🏷️ Аналіз ефективності A/B груп push-сповіщень

Цей ноутбук аналізує ефективність різних A/B груп push-сповіщень та їх вплив на депозити Android-користувачів.

### Вхідні дані:
- **6 A/B груп** (1, 2, 3, 4, 5, 6)
- **3,219,484 користувачів** загалом
- **41,193 користувачів з депозитами** (1.28%)
- **$226,815 загального доходу**

### Ключові питання:
1. Яка A/B група найефективніша?
2. Чи є статистично значуща різниця між групами?
3. Як кількість push-ів впливає на конверсію?
4. Які рекомендації для оптимізації?

---

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu
import warnings
warnings.filterwarnings('ignore')

# Налаштування візуалізації
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("🏷️ A/B АНАЛІЗ PUSH-ГРУП")
print("=" * 50)
print(f"🎯 Мета: Визначити найефективнішу A/B групу")
print(f"📊 Метрики: Конверсія, дохід, статистична значущість")

## **КРОК 1: Завантаження готових даних для A/B аналізу**

In [ ]:
print("\n🔗 КРОК 1: ЗАВАНТАЖЕННЯ ДАНИХ ДЛЯ A/B АНАЛІЗУ")
print("-" * 40)

# Завантажуємо готові дані
try:
    ab_df = pd.read_parquet('../data/processed/ab_analysis_ready.parquet')
    print(f"✅ Дані завантажено: {len(ab_df):,} записів")
except FileNotFoundError:
    print("❌ Файл ab_analysis_ready.parquet не знайдено")
    print("💡 Запустіть спочатку 03_data_matching.ipynb")
    raise

print(f"📊 Колонки: {list(ab_df.columns)}")
print(f"📏 Розмір датасету: {ab_df.shape}")

# Базова інформація
print("\n📈 БАЗОВА СТАТИСТИКА:")
print(f"   👥 Загалом користувачів: {len(ab_df):,}")
print(f"   📱 Загалом push-ів: {ab_df['total_pushes'].sum():,}")
print(f"   💰 Користувачі з депозитами: {ab_df['has_deposit'].sum():,}")
print(f"   💵 Загальний дохід: ${ab_df['total_revenue'].sum():,.2f}")

# A/B групи
print(f"\n🏷️ A/B ГРУПИ:")
group_counts = ab_df['ab_group'].value_counts().sort_index()
for group, count in group_counts.items():
    pct = count / len(ab_df) * 100
    print(f"   Група {group}: {count:,} користувачів ({pct:.1f}%)")

# Перевіряємо чи є дані по tier
if 'tier' in ab_df.columns:
    print(f"\n🌍 РОЗПОДІЛ ПО TIER:")
    tier_counts = ab_df['tier'].value_counts()
    for tier, count in tier_counts.head().items():
        print(f"   {tier}: {count:,} користувачів")

# Показуємо приклад
print("\n🔍 ПРИКЛАД ДАНИХ:")
display(ab_df.head())

## **КРОК 2: Базовий A/B аналіз по групах**

In [ ]:
print("\n🔗 КРОК 2: БАЗОВИЙ A/B АНАЛІЗ")
print("-" * 40)

# Агрегація по A/B групах
ab_summary = ab_df.groupby('ab_group').agg({
    'gadid': 'count',                    # Кількість користувачів
    'total_pushes': ['mean', 'median', 'std'],  # Push статистики
    'has_deposit': ['sum', 'mean'],       # Депозити
    'has_registration': ['sum', 'mean'], # Реєстрації
    'total_revenue': ['sum', 'mean']     # Дохід
}).round(4)

# Спрощуємо назви колонок
ab_summary.columns = [
    'users', 'avg_pushes', 'median_pushes', 'std_pushes',
    'total_deposits', 'deposit_rate', 'total_registrations', 'reg_rate',
    'total_revenue', 'avg_revenue_per_user'
]

# Додаємо розраховані метрики
ab_summary['deposit_rate_pct'] = ab_summary['deposit_rate'] * 100
ab_summary['reg_rate_pct'] = ab_summary['reg_rate'] * 100
ab_summary['revenue_per_push'] = ab_summary['total_revenue'] / (ab_summary['users'] * ab_summary['avg_pushes'])
ab_summary['arpu'] = ab_summary['total_revenue'] / ab_summary['users']

print("📊 ЗАГАЛЬНА СТАТИСТИКА ПО A/B ГРУПАХ:")
print(ab_summary[['users', 'avg_pushes', 'deposit_rate_pct', 'total_revenue', 'arpu']])

# Виділяємо найкращі результати
print("\n🏆 НАЙКРАЩІ ПОКАЗНИКИ:")
best_conversion = ab_summary['deposit_rate_pct'].idxmax()
best_revenue = ab_summary['total_revenue'].idxmax()
best_arpu = ab_summary['arpu'].idxmax()

print(f"   🎯 Найвища конверсія: Група {best_conversion} ({ab_summary.loc[best_conversion, 'deposit_rate_pct']:.2f}%)")
print(f"   💰 Найбільший дохід: Група {best_revenue} (${ab_summary.loc[best_revenue, 'total_revenue']:,.2f})")
print(f"   💵 Найвищий ARPU: Група {best_arpu} (${ab_summary.loc[best_arpu, 'arpu']:.2f})")

# Перевіряємо групу 6 (аномальна)
if '6' in ab_summary.index:
    group6_users = ab_summary.loc['6', 'users']
    group6_conv = ab_summary.loc['6', 'deposit_rate_pct']
    print(f"\n⚠️ УВАГА - Група 6: {group6_users} користувачів, {group6_conv:.1f}% конверсія")
    print(f"   💡 Мала група - результати можуть бути не репрезентативними")

# Статистика по push-ах
print("\n📱 СТАТИСТИКА PUSH-ІВ ПО ГРУПАХ:")
push_stats = ab_summary[['avg_pushes', 'median_pushes', 'std_pushes']].round(2)
print(push_stats)

# Кореляція push-ів та конверсії
correlation = ab_summary['avg_pushes'].corr(ab_summary['deposit_rate_pct'])
print(f"\n📈 Кореляція к-ть push-ів ↔ конверсія: {correlation:.3f}")
if correlation < -0.5:
    print(f"   💡 Сильна негативна кореляція - багато push-ів = гірша конверсія")
elif correlation > 0.5:
    print(f"   💡 Сильна позитивна кореляція - багато push-ів = краща конверсія")
else:
    print(f"   💡 Слабка кореляція - зв'язок неочевидний")

## **КРОК 3: Статистичні тести значущості**

In [ ]:
print("\n🔗 КРОК 3: СТАТИСТИЧНІ ТЕСТИ")
print("-" * 40)

# 1. Chi-square тест для конверсій
print("📊 CHI-SQUARE ТЕСТ ДЛЯ КОНВЕРСІЙ:")

# Створюємо contingency table
contingency_table = pd.crosstab(ab_df['ab_group'], ab_df['has_deposit'])
print("\nТаблиця спряженості:")
print(contingency_table)

# Виконуємо chi-square тест
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\n📈 Результати Chi-square тесту:")
print(f"   Chi-square статистика: {chi2:.4f}")
print(f"   P-value: {p_value:.6f}")
print(f"   Ступені свободи: {dof}")

alpha = 0.05
if p_value < alpha:
    print(f"   ✅ СТАТИСТИЧНО ЗНАЧУЩА РІЗНИЦЯ (p < {alpha})")
    print(f"   💡 A/B групи мають різні коефіцієнти конверсії")
else:
    print(f"   ❌ Немає статистично значущої різниці (p >= {alpha})")
    print(f"   💡 A/B групи не відрізняються суттєво")

# 2. Попарні порівняння основних груп (виключаємо групу 6)
print(f"\n🔍 ПОПАРНІ ПОРІВНЯННЯ ОСНОВНИХ ГРУП (1-5):")

main_groups = ['1', '2', '3', '4', '5']
ab_main = ab_df[ab_df['ab_group'].isin(main_groups)]

significant_pairs = []
for i, group1 in enumerate(main_groups):
    for group2 in main_groups[i+1:]:
        # Дані для двох груп
        g1_data = ab_main[ab_main['ab_group'] == group1]
        g2_data = ab_main[ab_main['ab_group'] == group2]
        
        # Chi-square для конверсій
        g1_conv = [g1_data['has_deposit'].sum(), len(g1_data) - g1_data['has_deposit'].sum()]
        g2_conv = [g2_data['has_deposit'].sum(), len(g2_data) - g2_data['has_deposit'].sum()]
        
        try:
            chi2_pair, p_pair = stats.chi2_contingency([g1_conv, g2_conv])[:2]
            
            if p_pair < 0.05:
                significant_pairs.append((group1, group2, p_pair))
                conv1 = g1_data['has_deposit'].mean() * 100
                conv2 = g2_data['has_deposit'].mean() * 100
                print(f"   Група {group1} vs {group2}: p={p_pair:.4f} ✅ ({conv1:.2f}% vs {conv2:.2f}%)")
        except:
            continue

if not significant_pairs:
    print(f"   💡 Немає статистично значущих різниць між основними групами")

# 3. Mann-Whitney тест для доходу
print(f"\n💰 MANN-WHITNEY ТЕСТ ДЛЯ ДОХОДУ:")

# Беремо користувачів з доходом > 0
revenue_users = ab_main[ab_main['total_revenue'] > 0]
if len(revenue_users) > 100:
    # Порівнюємо групи з найбільшою різницею в ARPU
    group_arpu = revenue_users.groupby('ab_group')['total_revenue'].mean()
    best_group = group_arpu.idxmax()
    worst_group = group_arpu.idxmin()
    
    best_revenue = revenue_users[revenue_users['ab_group'] == best_group]['total_revenue']
    worst_revenue = revenue_users[revenue_users['ab_group'] == worst_group]['total_revenue']
    
    statistic, p_revenue = mannwhitneyu(best_revenue, worst_revenue, alternative='two-sided')
    
    print(f"   Група {best_group} vs {worst_group}:")
    print(f"   Статистика: {statistic:.2f}, P-value: {p_revenue:.4f}")
    print(f"   Медіанний дохід: ${best_revenue.median():.2f} vs ${worst_revenue.median():.2f}")
    
    if p_revenue < 0.05:
        print(f"   ✅ Статистично значуща різниця в доходах")
    else:
        print(f"   ❌ Немає значущої різниці в доходах")
else:
    print(f"   ⚠️ Недостатньо даних для тестування ({len(revenue_users)} користувачів з доходом)")

## **КРОК 4: Візуалізації A/B результатів**

In [ ]:
print("\n🔗 КРОК 4: ВІЗУАЛІЗАЦІЇ")
print("-" * 40)

# Створюємо фігуру з субплотами
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('A/B Аналіз Push-груп: Ключові метрики', fontsize=16, fontweight='bold')

# 1. Конверсія по групах
ax1 = axes[0, 0]
conversion_rates = ab_summary['deposit_rate_pct']
bars1 = ax1.bar(conversion_rates.index, conversion_rates.values, 
                color=sns.color_palette("husl", len(conversion_rates)))
ax1.set_title('Коефіцієнт конверсії push → депозит (%)')
ax1.set_xlabel('A/B Група')
ax1.set_ylabel('Конверсія (%)')
ax1.grid(True, alpha=0.3)

# Додаємо значення на стовпці
for bar, value in zip(bars1, conversion_rates.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.2f}%', ha='center', va='bottom')

# 2. Загальний дохід по групах
ax2 = axes[0, 1]
revenues = ab_summary['total_revenue']
bars2 = ax2.bar(revenues.index, revenues.values, 
                color=sns.color_palette("viridis", len(revenues)))
ax2.set_title('Загальний дохід по групах ($)')
ax2.set_xlabel('A/B Група')
ax2.set_ylabel('Дохід ($)')
ax2.grid(True, alpha=0.3)

# Форматуємо як валюту
for bar, value in zip(bars2, revenues.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100,
             f'${value:,.0f}', ha='center', va='bottom', rotation=45)

# 3. Середня кількість push-ів
ax3 = axes[0, 2]
avg_pushes = ab_summary['avg_pushes']
bars3 = ax3.bar(avg_pushes.index, avg_pushes.values,
                color=sns.color_palette("plasma", len(avg_pushes)))
ax3.set_title('Середня кількість push-ів на користувача')
ax3.set_xlabel('A/B Група')
ax3.set_ylabel('Кількість push-ів')
ax3.grid(True, alpha=0.3)

for bar, value in zip(bars3, avg_pushes.values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{value:.1f}', ha='center', va='bottom')

# 4. ARPU по групах
ax4 = axes[1, 0]
arpu_values = ab_summary['arpu']
bars4 = ax4.bar(arpu_values.index, arpu_values.values,
                color=sns.color_palette("coolwarm", len(arpu_values)))
ax4.set_title('ARPU (Average Revenue Per User)')
ax4.set_xlabel('A/B Група')
ax4.set_ylabel('ARPU ($)')
ax4.grid(True, alpha=0.3)

for bar, value in zip(bars4, arpu_values.values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'${value:.3f}', ha='center', va='bottom')

# 5. Розподіл користувачів по групах
ax5 = axes[1, 1]
user_counts = ab_summary['users']
bars5 = ax5.bar(user_counts.index, user_counts.values,
                color=sns.color_palette("Set2", len(user_counts)))
ax5.set_title('Кількість користувачів по групах')
ax5.set_xlabel('A/B Група')
ax5.set_ylabel('Кількість користувачів')
ax5.grid(True, alpha=0.3)

for bar, value in zip(bars5, user_counts.values):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5000,
             f'{value:,}', ha='center', va='bottom', rotation=45, fontsize=8)

# 6. Scatter plot: Push-и vs Конверсія
ax6 = axes[1, 2]
# Виключаємо групу 6 якщо вона аномальна
plot_data = ab_summary.copy()
if '6' in plot_data.index and plot_data.loc['6', 'users'] < 1000:
    plot_data = plot_data.drop('6')

scatter = ax6.scatter(plot_data['avg_pushes'], plot_data['deposit_rate_pct'], 
                     s=plot_data['users']/1000, alpha=0.7,
                     c=range(len(plot_data)), cmap='tab10')
ax6.set_title('Push-и vs Конверсія (розмір = к-ть користувачів)')
ax6.set_xlabel('Середня кількість push-ів')
ax6.set_ylabel('Конверсія (%)')
ax6.grid(True, alpha=0.3)

# Додаємо лейбли груп
for idx, (group, row) in enumerate(plot_data.iterrows()):
    ax6.annotate(f'Група {group}', 
                (row['avg_pushes'], row['deposit_rate_pct']),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

print("✅ Візуалізації створено")
print("💡 Зверніть увагу на залежність між кількістю push-ів та конверсією")

## **КРОК 5: Глибокий аналіз сегментів**

In [ ]:
print("\n🔗 КРОК 5: ГЛИБОКИЙ АНАЛІЗ СЕГМЕНТІВ")
print("-" * 40)

# 1. Аналіз по сегментах push-ів
if 'push_segment' in ab_df.columns:
    print("📱 АНАЛІЗ ПО СЕГМЕНТАХ PUSH-ІВ:")
    
    segment_analysis = ab_df.groupby(['ab_group', 'push_segment']).agg({
        'gadid': 'count',
        'has_deposit': 'mean',
        'total_revenue': 'sum'
    }).round(4)
    
    segment_analysis.columns = ['users', 'conversion_rate', 'total_revenue']
    segment_analysis['conversion_pct'] = segment_analysis['conversion_rate'] * 100
    
    # Показуємо топ сегменти по конверсії
    top_segments = segment_analysis[segment_analysis['users'] >= 1000].nlargest(10, 'conversion_pct')
    print("\nТоп-10 сегментів по конверсії (мін. 1000 користувачів):")
    for (group, segment), row in top_segments.iterrows():
        print(f"   Група {group}, {segment} push-ів: {row['conversion_pct']:.2f}% конверсія, {row['users']} користувачів")

# 2. Аналіз по географічних tier-ах
if 'tier' in ab_df.columns:
    print("\n🌍 АНАЛІЗ ПО ГЕОГРАФІЧНИХ TIER-АХ:")
    
    tier_analysis = ab_df.groupby(['ab_group', 'tier']).agg({
        'gadid': 'count',
        'has_deposit': 'mean',
        'total_revenue': 'sum'
    }).round(4)
    
    tier_analysis.columns = ['users', 'conversion_rate', 'total_revenue']
    tier_analysis['conversion_pct'] = tier_analysis['conversion_rate'] * 100
    
    # Аналіз по Tier 1 та Tier 2 (найвагоміші)
    for tier in ['Tier 1', 'Tier 2']:
        if tier in ab_df['tier'].values:
            tier_data = tier_analysis.loc[tier_analysis.index.get_level_values(1) == tier]
            if not tier_data.empty:
                print(f"\n{tier}:")
                tier_data_sorted = tier_data.sort_values('conversion_pct', ascending=False)
                for group, row in tier_data_sorted.iterrows():
                    if isinstance(group, tuple):
                        group = group[0]
                    print(f"   Група {group}: {row['conversion_pct']:.2f}% конверсія, {row['users']:,} користувачів")

# 3. Аналіз категорій користувачів
if 'user_category' in ab_df.columns:
    print("\n👥 РОЗПОДІЛ КАТЕГОРІЙ ПО A/B ГРУПАХ:")
    
    category_crosstab = pd.crosstab(ab_df['ab_group'], ab_df['user_category'], normalize='index') * 100
    print(category_crosstab.round(2))
    
    # Знаходимо групу з найвищим відсотком депозиторів
    if 'Depositor' in category_crosstab.columns:
        best_depositor_group = category_crosstab['Depositor'].idxmax()
        best_depositor_pct = category_crosstab.loc[best_depositor_group, 'Depositor']
        print(f"\n🏆 Найвищий % депозиторів: Група {best_depositor_group} ({best_depositor_pct:.2f}%)")

# 4. Аналіз віддачі (ROI) по групах
print("\n💰 АНАЛІЗ ВІДДАЧІ (ROI):")

# Припускаємо умовну вартість push-а
cost_per_push = 0.01  # $0.01 за push

roi_analysis = ab_summary.copy()
roi_analysis['total_cost'] = roi_analysis['users'] * roi_analysis['avg_pushes'] * cost_per_push
roi_analysis['roi'] = (roi_analysis['total_revenue'] - roi_analysis['total_cost']) / roi_analysis['total_cost'] * 100
roi_analysis['profit'] = roi_analysis['total_revenue'] - roi_analysis['total_cost']

print(f"ROI аналіз (при вартості ${cost_per_push} за push):")
roi_display = roi_analysis[['total_revenue', 'total_cost', 'profit', 'roi']].round(2)
print(roi_display)

best_roi_group = roi_analysis['roi'].idxmax()
best_roi_value = roi_analysis.loc[best_roi_group, 'roi']
print(f"\n🏆 Найкращий ROI: Група {best_roi_group} ({best_roi_value:.1f}%)")

# Виключаємо групу 6 якщо вона аномальна
if '6' in roi_analysis.index and roi_analysis.loc['6', 'users'] < 1000:
    main_roi = roi_analysis.drop('6')
    best_main_roi = main_roi['roi'].idxmax()
    best_main_roi_value = main_roi.loc[best_main_roi, 'roi']
    print(f"🏆 Найкращий ROI (основні групи): Група {best_main_roi} ({best_main_roi_value:.1f}%)")

## **КРОК 6: Оптимальна стратегія push-ів**

In [ ]:
print("\n🔗 КРОК 6: ОПТИМАЛЬНА СТРАТЕГІЯ")
print("-" * 40)

# Аналіз оптимальної кількості push-ів
print("🎯 ОПТИМАЛЬНА КІЛЬКІСТЬ PUSH-ІВ:")

# Групуємо по кількості push-ів (діапазони)
ab_df['push_range'] = pd.cut(ab_df['total_pushes'], 
                            bins=[0, 10, 20, 30, 50, 100, float('inf')],
                            labels=['1-10', '11-20', '21-30', '31-50', '51-100', '100+'])

push_range_analysis = ab_df.groupby('push_range').agg({
    'gadid': 'count',
    'has_deposit': 'mean',
    'total_revenue': 'sum'
}).round(4)

push_range_analysis.columns = ['users', 'conversion_rate', 'total_revenue']
push_range_analysis['conversion_pct'] = push_range_analysis['conversion_rate'] * 100
push_range_analysis['revenue_per_user'] = push_range_analysis['total_revenue'] / push_range_analysis['users']

print("\nКонверсія по діапазонах push-ів:")
for range_name, row in push_range_analysis.iterrows():
    print(f"   {range_name} push-ів: {row['conversion_pct']:.2f}% конверсія, "
          f"{row['users']:,} користувачів, ${row['revenue_per_user']:.3f} доходу на користувача")

# Знаходимо оптимальний діапазон
optimal_range = push_range_analysis['conversion_pct'].idxmax()
optimal_conversion = push_range_analysis.loc[optimal_range, 'conversion_pct']
print(f"\n🏆 Оптимальний діапазон: {optimal_range} push-ів ({optimal_conversion:.2f}% конверсія)")

# Аналіз найкращої A/B групи в оптимальному діапазоні
optimal_data = ab_df[ab_df['push_range'] == optimal_range]
if len(optimal_data) > 100:
    optimal_ab = optimal_data.groupby('ab_group').agg({
        'gadid': 'count',
        'has_deposit': 'mean'
    })
    optimal_ab['conversion_pct'] = optimal_ab['has_deposit'] * 100
    
    print(f"\nКонверсія по A/B групах в діапазоні {optimal_range}:")
    for group, row in optimal_ab.sort_values('conversion_pct', ascending=False).iterrows():
        if row['gadid'] >= 50:  # Мінімум 50 користувачів
            print(f"   Група {group}: {row['conversion_pct']:.2f}% ({row['gadid']} користувачів)")

# Рекомендації по частоті
print("\n📅 РЕКОМЕНДАЦІЇ ПО ЧАСТОТІ:")

# Аналізуємо pushes_per_day якщо доступно
if 'pushes_per_day' in ab_df.columns:
    frequency_analysis = ab_df.groupby('ab_group').agg({
        'pushes_per_day': ['mean', 'median'],
        'has_deposit': 'mean'
    }).round(3)
    
    frequency_analysis.columns = ['avg_pushes_per_day', 'median_pushes_per_day', 'conversion_rate']
    frequency_analysis['conversion_pct'] = frequency_analysis['conversion_rate'] * 100
    
    print("\nЧастота push-ів по групах:")
    for group, row in frequency_analysis.iterrows():
        print(f"   Група {group}: {row['avg_pushes_per_day']:.1f} push/день, "
              f"{row['conversion_pct']:.2f}% конверсія")
    
    # Кореляція частоти та конверсії
    freq_corr = frequency_analysis['avg_pushes_per_day'].corr(frequency_analysis['conversion_pct'])
    print(f"\n📈 Кореляція частота ↔ конверсія: {freq_corr:.3f}")
    
    if freq_corr < -0.3:
        print(f"   💡 Рекомендація: Зменшити частоту push-ів для кращої конверсії")
    elif freq_corr > 0.3:
        print(f"   💡 Рекомендація: Можна збільшити частоту push-ів")
    else:
        print(f"   💡 Частота push-ів не сильно впливає на конверсію")

# Підсумкові рекомендації
print("\n💡 ПІДСУМКОВІ РЕКОМЕНДАЦІЇ:")

# Найкраща група для основного трафіку
main_groups_summary = ab_summary.copy()
if '6' in main_groups_summary.index and main_groups_summary.loc['6', 'users'] < 1000:
    main_groups_summary = main_groups_summary.drop('6')

best_overall = main_groups_summary['deposit_rate_pct'].idxmax()
best_revenue = main_groups_summary['total_revenue'].idxmax()
best_efficiency = main_groups_summary['arpu'].idxmax()

print(f"   🏆 Найкраща конверсія: Група {best_overall} ({main_groups_summary.loc[best_overall, 'deposit_rate_pct']:.2f}%)")
print(f"   💰 Найбільший дохід: Група {best_revenue} (${main_groups_summary.loc[best_revenue, 'total_revenue']:,.0f})")
print(f"   ⚡ Найефективніша: Група {best_efficiency} (${main_groups_summary.loc[best_efficiency, 'arpu']:.3f} ARPU)")
print(f"   📱 Оптимальна к-ть push-ів: {optimal_range}")

if best_overall == best_revenue == best_efficiency:
    print(f"\n🎯 ОДНОЗНАЧНА РЕКОМЕНДАЦІЯ: Використовувати Групу {best_overall}")
else:
    print(f"\n🎯 РЕКОМЕНДАЦІЯ: Група {best_overall} для максимальної конверсії")
    print(f"   💡 Альтернатива: Група {best_revenue} для максимального доходу")

## **КРОК 7: Збереження результатів та висновків**

In [ ]:
print("\n🔗 КРОК 7: ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ")
print("-" * 40)

# Створюємо підсумковий звіт
ab_report = {
    'analysis_info': {
        'total_users': len(ab_df),
        'total_deposits': ab_df['has_deposit'].sum(),
        'overall_conversion_rate': ab_df['has_deposit'].mean() * 100,
        'total_revenue': ab_df['total_revenue'].sum(),
        'analysis_date': pd.Timestamp.now().isoformat()
    },
    'ab_group_performance': {},
    'statistical_tests': {
        'chi_square_p_value': p_value,
        'significant_difference': p_value < 0.05
    },
    'recommendations': {
        'best_conversion_group': best_overall,
        'best_revenue_group': best_revenue,
        'optimal_push_range': optimal_range,
        'push_conversion_correlation': correlation
    }
}

# Заповнюємо дані по групах
for group, row in ab_summary.iterrows():
    ab_report['ab_group_performance'][f'group_{group}'] = {
        'users': int(row['users']),
        'avg_pushes': float(row['avg_pushes']),
        'deposit_rate_pct': float(row['deposit_rate_pct']),
        'total_revenue': float(row['total_revenue']),
        'arpu': float(row['arpu'])
    }

# Зберігаємо звіт
import json
os.makedirs('../outputs/tables', exist_ok=True)

with open('../outputs/tables/ab_analysis_report.json', 'w', encoding='utf-8') as f:
    json.dump(ab_report, f, ensure_ascii=False, indent=2)

# Зберігаємо детальну таблицю
ab_summary_extended = ab_summary.copy()
ab_summary_extended.to_csv('../outputs/tables/ab_groups_detailed_analysis.csv')

# Зберігаємо візуалізацію
plt.savefig('../outputs/charts/ab_analysis_summary.png', dpi=300, bbox_inches='tight')

print("✅ Звіт збережено: ab_analysis_report.json")
print("✅ Детальна таблиця: ab_groups_detailed_analysis.csv")
print("✅ Візуалізація: ab_analysis_summary.png")

# Фінальний підсумок
print("\n" + "="*60)
print("📋 ФІНАЛЬНИЙ ПІДСУМОК A/B АНАЛІЗУ")
print("="*60)

print(f"📊 Проаналізовано {len(ab_df):,} користувачів у {ab_df['ab_group'].nunique()} A/B групах")
print(f"💰 Загальна конверсія: {ab_df['has_deposit'].mean()*100:.2f}%")
print(f"💵 Загальний дохід: ${ab_df['total_revenue'].sum():,.2f}")

if p_value < 0.05:
    print(f"📈 Статистично значуща різниця між групами (p={p_value:.4f})")
else:
    print(f"📈 Немає статистично значущої різниці (p={p_value:.4f})")

print(f"\n🏆 РЕКОМЕНДОВАНА СТРАТЕГІЯ:")
print(f"   🎯 Використовувати Групу {best_overall} для найкращої конверсії")
print(f"   📱 Оптимальна кількість push-ів: {optimal_range}")
print(f"   💡 Кореляція push-ів та конверсії: {correlation:.3f}")

if correlation < -0.3:
    print(f"   ⚠️ Увага: Більше push-ів = гірша конверсія. Рекомендується зменшити частоту")
elif correlation > 0.3:
    print(f"   ✅ Більше push-ів = краща конверсія. Можна збільшити частоту")

print(f"\n➡️ Наступний крок: 05_geo_tier_analysis.ipynb для географічного аналізу")
print(f"🚀 A/B АНАЛІЗ ЗАВЕРШЕНО!")

## **Висновки A/B аналізу**

### ✅ **Ключові результати:**
1. **Статистична значущість** - визначено чи є реальна різниця між A/B групами
2. **Оптимальна стратегія** - знайдено найефективнішу групу та кількість push-ів
3. **Кореляційний аналіз** - з'ясовано зв'язок між частотою push-ів та конверсією
4. **Сегментний аналіз** - проаналізовано ефективність по tier-ах та категоріях

### 📊 **Методологія:**
- Chi-square тест для порівняння конверсій
- Mann-Whitney тест для порівняння доходів
- Кореляційний аналіз push-ів та результатів
- ROI аналіз з урахуванням умовної вартості push-ів

### 🎯 **Практичні рекомендації:**
Визначено оптимальну A/B групу, частоту push-сповіщень та стратегію для максимізації конверсії та доходу.

### ➡️ **Наступні кроки:**
Географічний аналіз ефективності по tier-ах країн для локалізації стратегії.